In [7]:
%load_ext wurlitzer
import os
os.environ['CC'] = '/Library/Developer/CommandLineTools/usr/bin/clang'
os.environ['CXX'] = '/Library/Developer/CommandLineTools/usr/bin/clang++'

!which clang
import time

import numpy as np
from matplotlib import pyplot as plt
import xobjects as xo
import xtrack as xt
import xfields as xf
print(xf.__file__)
import xpart as xp



#L0 with combi formula

import time

import numpy as np
from matplotlib import pyplot as plt
import xobjects as xo
import xtrack as xt
import xfieldsdev as xf
import xpart as xp

###################
#To run this code:
#mpirun -np 2 python xsuite_dev/xtrack/examples/L0beambeam.py
lumi_qss_b1 = []
lumi_averages_b1 = []

lumi_qss_b1_nobeambeam = []
lumi_averages_b1_nobeambeam = []


context = xo.ContextCpu(omp_num_threads=0)


#################################
# Generate particles            #
#################################

p0c = 7e12
n_macroparticles = int(1.2e4)
bunch_intensity = 2.5e11 #7.8e10
physemit_x = (2E-6*xp.PROTON_MASS_EV)/p0c #(2.95E-6*xp.PROTON_MASS_EV)/p0c
physemit_y = (2E-6*xp.PROTON_MASS_EV)/p0c #(2.95E-6*xp.PROTON_MASS_EV)/p0c
beta_x_IP1 = 19.2
beta_y_IP1 = 19.2
sigma_z = 0.08
sigma_delta = 1E-4
beta_s = sigma_z/sigma_delta
Qx = 63.31
Qy = 60.32
Qs = 2.1E-3
frev = 11245.5 
nTurn = 100


colors = ['b', 'g', 'r', 'c', 'm', 'k', 'y']
xshift = [0, 1, 2, 3, 4, 5, 6]
yshift = [0, 1, 2, 3, 4, 5, 6]

for shift in xshift:
    context = xo.ContextCpu(omp_num_threads=0)

    pipeline_manager = xt.PipelineManager()
    pipeline_manager.add_particles('b1',0)
    pipeline_manager.add_particles('b2',0)
    pipeline_manager.add_element('IP1')

    particles_b1 = xp.Particles(_context=context,
        p0c=p0c,
        x=np.sqrt(physemit_x*beta_x_IP1)*(np.random.randn(n_macroparticles)),
        px=np.sqrt(physemit_x/beta_x_IP1)*np.random.randn(n_macroparticles),
        y=np.sqrt(physemit_y*beta_y_IP1)*(np.random.randn(n_macroparticles)),
        py=np.sqrt(physemit_y/beta_y_IP1)*np.random.randn(n_macroparticles),
        zeta=sigma_z*np.random.randn(n_macroparticles),
        delta=sigma_delta*np.random.randn(n_macroparticles),
        weight=bunch_intensity/n_macroparticles
    )
    particles_b1.init_pipeline('b1')
    particles_b2 = xp.Particles(_context=context,
        p0c=p0c,
        x=np.sqrt(physemit_x*beta_x_IP1)*(np.random.randn(n_macroparticles)),
        px=np.sqrt(physemit_x/beta_x_IP1)*np.random.randn(n_macroparticles),
        y=np.sqrt(physemit_y*beta_y_IP1)*(np.random.randn(n_macroparticles)),
        py=np.sqrt(physemit_y/beta_y_IP1)*np.random.randn(n_macroparticles),
        zeta=sigma_z*np.random.randn(n_macroparticles),
        delta=sigma_delta*np.random.randn(n_macroparticles),
        weight=bunch_intensity/n_macroparticles
    )
    particles_b2.init_pipeline('b2')

    #############
    # Beam-beam #
    #############
    slicer = xf.TempSlicer(sigma_z=sigma_z, n_slices=1, mode = 'shatilov')
    config_for_update_b1_IP1=xf.ConfigForUpdateBeamBeamBiGaussian3D(
    pipeline_manager=pipeline_manager,
    element_name='IP1',
    partner_particles_name = 'b2',
    slicer=slicer,
    update_every=1,
    n_lumigrid_cells = 24*24
    )
    config_for_update_b2_IP1=xf.ConfigForUpdateBeamBeamBiGaussian3D(
    pipeline_manager=pipeline_manager,
    element_name='IP1',
    partner_particles_name = 'b1',
    slicer=slicer,
    update_every=1,
    n_lumigrid_cells=24*24
    )

    print('build bb elements...')
    bbeamIP1_b1 = xf.BeamBeamBiGaussian3D(
                _context=context,
                other_beam_q0 = particles_b2.q0,
                phi = 0,alpha=0,
                config_for_update = config_for_update_b1_IP1,
                ref_shift_x = shift*np.sqrt(physemit_x*beta_x_IP1)/2,
                flag_luminosity=1,
                flag_combilumi=1,
                beam_intensity=bunch_intensity,
                other_beam_intensity=bunch_intensity)
    bbeamIP1_b2 = xf.BeamBeamBiGaussian3D(
                _context=context,
                other_beam_q0 = particles_b1.q0,
                phi = 0,alpha=0,
                config_for_update = config_for_update_b2_IP1,
                ref_shift_x = shift*np.sqrt(physemit_x*beta_x_IP1)/2)



    #################################################################
    # arcs (here they are all the same with half the phase advance) #
    #################################################################

    arc = xt.LineSegmentMap(
            betx = beta_x_IP1,bety = beta_y_IP1,
            qx = Qx, qy = Qy,bets = beta_s, qs=Qs)

    #################################################################
    # Tracker                                                       #
    #################################################################

    elements_b1 = [bbeamIP1_b1,arc]
    elements_b2 = [bbeamIP1_b2,arc]
    line_b1 = xt.Line(elements=elements_b1)
    line_b2 = xt.Line(elements=elements_b2)
    line_b1.build_tracker()
    line_b2.build_tracker()
    branch_b1 = xt.PipelineBranch(line_b1,particles_b1)
    branch_b2 = xt.PipelineBranch(line_b2,particles_b2)
    multitracker = xt.PipelineMultiTracker(branches=[branch_b1,branch_b2])

    #################################################################
    # Tracking                                                      #
    #################################################################

    record_qss_b1 = line_b1.start_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D, 
                                                            capacity={
                                                                "beamstrahlungtable": int(0),
                                                                "bhabhatable": int(0),
                                                                "lumitable": nTurn
                                                            })


    print('Tracking...')
    time0 = time.time()

    multitracker.track(num_turns=nTurn,turn_by_turn_monitor=True)
    print('Done with tracking.',(time.time()-time0)/16384,'[s/turn]')
    line_b1.stop_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D)

    record_qss_b1.move(_context=xo.context_default)
 
    lumi_b1_nobeambeam = record_qss_b1.lumitable.luminosity
    
    lumi_qss_b1_nobeambeam.append(lumi_b1_nobeambeam)

    lumi_averages_b1_nobeambeam.append(np.mean(lumi_b1_nobeambeam))


lumis = []
separation = [0, 1, 2, 3, 4, 5, 6]

def Lumi_analytical(Nb, N1, N2, frev, Delta_i, sig_i, sig_x, sig_y):
    W = np.exp(-Delta_i**2/(4*sig_i**2))
    return ((Nb * N1 * N2 * frev * W)/(4 * np.pi * 100 * sig_x * 100 * sig_y))

for i in range(len(separation)):
    lumis.append(Lumi_analytical(n_macroparticles, bunch_intensity, bunch_intensity, frev, separation[i]*np.sqrt(physemit_x*beta_x_IP1),np.sqrt(physemit_x*beta_x_IP1), np.sqrt(physemit_x*beta_x_IP1), np.sqrt(physemit_y*beta_x_IP1)))

fig0, ax1 = plt.subplots()
fig1, ax2 = plt.subplots()

ax1.set_title("Luminosity as a function of beam separation")
ax1.set_xlabel("Separation (sigma)")
ax1.set_ylabel("Luminosity")
ax1.plot(separation, lumis, label = "Analytical")
ax1.plot(xshift, (frev*np.array(lumi_averages_b1)), label = "With beam beam") #Here i am confused- this is in m^-2 or cm^-2
ax1.plot(xshift, (frev*np.array(lumi_averages_b1_nobeambeam)), label = "Without beam beam")
ax1.legend()


ax2.set_title("L/L0")
ax2.plot(xshift, np.array(lumi_averages_b1)/np.array(lumi_averages_b1_nobeambeam))
plt.show()


The wurlitzer extension is already loaded. To reload it, use:
  %reload_ext wurlitzer
/usr/bin/clang
/Users/chenying/miniforge3/envs/xsuite-dev/lib/python3.11/site-packages/xfields/__init__.py
Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/chenying/miniforge3/envs/xsuite-dev/lib' ignored
ld: warning: duplicate -rpath '/Users/chenying/miniforge3/envs/xsuite-dev/lib' ignored
ld: warning: -pie being ignored. It is only used when linking a main executable
ld: warning: ignoring file '/Users/chenying/xsuite_dev/xtrack/xfields/xfieldsdev/beam_elements/beambeam_src/121c6f15fd964ceba832c7bbfd0a090b.o': found architecture 'arm64', required architecture 'x86_64'


Done compiling ContextCpu kernels.


ImportError: dlopen(/Users/chenying/xsuite_dev/xtrack/xfields/xfieldsdev/beam_elements/beambeam_src/121c6f15fd964ceba832c7bbfd0a090b.cpython-311-darwin.so, 0x0002): tried: '/Users/chenying/xsuite_dev/xtrack/xfields/xfieldsdev/beam_elements/beambeam_src/121c6f15fd964ceba832c7bbfd0a090b.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/chenying/xsuite_dev/xtrack/xfields/xfieldsdev/beam_elements/beambeam_src/121c6f15fd964ceba832c7bbfd0a090b.cpython-311-darwin.so' (no such file), '/Users/chenying/xsuite_dev/xtrack/xfields/xfieldsdev/beam_elements/beambeam_src/121c6f15fd964ceba832c7bbfd0a090b.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))

In [ ]:
%load_ext wurlitzer

import time

import numpy as np
from matplotlib import pyplot as plt
import xobjects as xo
import xtrack as xt
import xfieldsdev as xf
print(xf.__file__)
import xpart as xp



#testing
###################
#To run this code:
#mpirun -np 2 python xsuite_dev/xtrack/examples/L0beambeam.py
lumi_qss_b1 = []
lumi_averages_b1 = []

combilumi_qss_b1 = []
combilumi_averages_b1 = []

lumi_qss_b1_nobeambeam = []
lumi_averages_b1_nobeambeam = []

combilumi_qss_b1_nobeambeam = []
combilumi_averages_b1_nobeambeam = []


context = xo.ContextCpu(omp_num_threads=0)


#################################
# Generate particles            #
#################################

p0c = 7e12
n_macroparticles = int(1.2e4)
bunch_intensity = 2.5e11 #7.8e10
physemit_x = (2E-6*xp.PROTON_MASS_EV)/p0c #(2.95E-6*xp.PROTON_MASS_EV)/p0c
physemit_y = (2E-6*xp.PROTON_MASS_EV)/p0c #(2.95E-6*xp.PROTON_MASS_EV)/p0c
beta_x_IP1 = 19.2
beta_y_IP1 = 19.2
sigma_z = 0.08
sigma_delta = 1E-4
beta_s = sigma_z/sigma_delta
Qx = 63.31
Qy = 60.32
Qs = 2.1E-3
frev = 11245.5 


colors = ['b', 'g', 'r', 'c', 'm', 'k', 'y']
xshift = [0, 1, 2, 3, 4, 5, 6]
yshift = [0, 1, 2, 3, 4, 5, 6]

for i in range(len(xshift)):
    context = xo.ContextCpu(omp_num_threads=0)

####################
# Pipeline manager #
####################

# Retrieving MPI info


# Building the pipeline manager based on a MPI communicator
    pipeline_manager = xt.PipelineManager()

    # Add information about the Particles instance that require
    # communication through the pipeline manager.
    # Each Particles instance must be identifiable by a unique name
    # The second argument is the MPI rank in which the Particles object
    # lives.
    pipeline_manager.add_particles('B1b1',0)
    pipeline_manager.add_particles('B2b1',1)
    # Add information about the elements that require communication
    # through the pipeline manager.
    # Each Element instance must be identifiable by a unique name
    # All Elements are instanciated in all ranks
    pipeline_manager.add_element('IP1')

    print('Initialising particles')
    particles = xp.Particles(_context=context,
        p0c=p0c,
        x=np.sqrt(physemit_x*beta_x_IP1)
            *(np.random.randn(n_macroparticles)),
        px=np.sqrt(physemit_x/beta_x_IP1)
            *np.random.randn(n_macroparticles),
        y=np.sqrt(physemit_y*beta_y_IP1)
            *(np.random.randn(n_macroparticles)),
        py=np.sqrt(physemit_y/beta_y_IP1)
            *np.random.randn(n_macroparticles),
        zeta=sigma_z*np.random.randn(n_macroparticles),
        delta=sigma_delta*np.random.randn(n_macroparticles),
        weight=bunch_intensity/n_macroparticles
    )
    # Initialise the Particles object with its unique name
    # (must match the info provided to the pipeline manager)
    particles.init_pipeline('B1b1')
    # Keep in memory the name of the Particles object with which this
    # rank will communicate
    # (must match the info provided to the pipeline manager)
    partner_particles_name = 'B2b1'

#############
# Beam-beam #
#############

    nb_slice = 1 #normally this is 11
    slicer = xf.TempSlicer(sigma_z=sigma_z, n_slices=nb_slice, mode = "shatilov") #or shatilov
    config_for_update_IP1=xf.ConfigForUpdateBeamBeamBiGaussian3D(
        pipeline_manager=pipeline_manager,
        element_name='IP1', # The element name must be unique and match the
                       # one given to the pipeline manager
        partner_particles_name = partner_particles_name, # the name of the
                       # Particles object with which to collide
        slicer=slicer,
        update_every=10,
        #update_every=1 # Setup for strong-strong simulation
        )



    print('build bb elements...')
    bbeamIP1 = xf.BeamBeamBiGaussian3D(
                _context=context,
                other_beam_q0 = particles.q0,
                phi = 0,alpha=0.0,
                config_for_update = config_for_update_IP1,
                ref_shift_x = xshift[i]*np.sqrt(physemit_x*beta_x_IP1),
                ref_shift_y = yshift[i]*np.sqrt(physemit_y*beta_y_IP1),
                flag_luminosity = 1,
                flag_combilumi = 1)

#################################################################
# arcs (here they are all the same with half the phase advance) #
#################################################################

    arc12 = xt.LineSegmentMap(
            betx = beta_x_IP1,bety = beta_y_IP1,
            qx = Qx/2, qy = Qy/2,bets = beta_s, qs=Qs/2)

    arc21 = xt.LineSegmentMap(
            betx = beta_x_IP1,bety = beta_y_IP1,
            qx = Qx/2, qy = Qy/2,bets = beta_s, qs=Qs/2)

#################################################################
# Tracker                                                       #
#################################################################

# In this example there is one Particles object per rank
# We build its line and tracker as usual
    elements = [arc12,bbeamIP1,arc21]
    line = xt.Line(elements=elements)
    line.build_tracker()
# A pipeline branch is a line and Particles object that will
# be tracked through it
    branch = xt.PipelineBranch(line, particles)
# The multitracker can deal with a set of branches (here only one)
    multitracker = xt.PipelineMultiTracker(branches=[branch])
    x = context.nparray_from_context_array(particles.x)
    y = context.nparray_from_context_array(particles.y)
    print(np.mean(x))
    print(np.mean(y))
    num_turns= 1000
    record_qss_b1 = line.start_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D, 
                                                                capacity={
                                                                    "beamstrahlungtable": int(0),
                                                                    "bhabhatable": int(0),
                                                                    "lumitable": num_turns,
                                                                    "combilumitable": num_turns,
                                                                    })
                                                                    #
                                                                


    multitracker.track(num_turns=num_turns)
    line.stop_internal_logging_for_elements_of_type(xf.BeamBeamBiGaussian3D)

    record_qss_b1.move(_context=xo.context_default)

 
    lumi_b1 = record_qss_b1.lumitable.luminosity
    combilumi_b1 = record_qss_b1.combilumitable.combilumi
    
    lumi_qss_b1.append(lumi_b1)
    combilumi_qss_b1.append(lumi_b1)

    lumi_averages_b1.append(np.mean(lumi_b1[200:]))
    combilumi_averages_b1.append(np.mean(lumi_b1[200:]))
    